In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

## 1. Scraping the Data from the Wikipedia Page

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html5lib')

#### 1.1 - Cleaning and wrangling the data

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

#### 1.2 - Checking for not assigned data

In [5]:
print(f"There are {df.loc[df['Borough'] == 'Not assigned'].shape[0]} Not assigned Boroughs")

There are 0 Not assigned Boroughs


In [6]:
print(f"There are {df.loc[df['Neighborhood'] == 'Not assigned'].shape[0]} Not assigned Neighborhoods")

There are 0 Not assigned Neighborhoods


In [11]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


#### 1.4 - Printing the shape of the data

In [14]:
df.shape

(103, 3)

## 2. Getting the geographical coordinates from the data

In [18]:
!pip install geocoder
import geocoder

for index, content in df.iterrows():
     coordinates = None
     while(coordinates is None):
         coordinates = geocoder.arcgis(f'{content[0]}, Toronto, Ontario').latlng
     latitude = coordinates[0]
     longitude = coordinates[1]
     df.loc[index, 'Latitude'] = latitude
     df.loc[index, 'Longitude'] = longitude

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 7.7 MB/s  eta 0:00:01


In [24]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


## 3. Explore and cluster the neighborhoods in Toronto

#### 3.1 - Creating map of all the city with neighborhoods superimposed on top

Getting the location of toronto with geopy

In [25]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto-explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [26]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### 3.2 - Selecting only the boroughs that contain the word Toronto

In [27]:
df_toronto_borough = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_borough.shape

(39, 5)

#### 3.3 - Creating map of toronto borough with neighborhoods superimposed on top

In [28]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_borough['Latitude'], df_toronto_borough['Longitude'], df_toronto_borough['Borough'], df_toronto_borough['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### 3.4. Use the Foursquare API to explore the neighborhoods

In [38]:
CLIENT_ID = 'PHSUFWQK30TO51RXWQO44GPOYGK1JEIVM1DKN1MRGNKOTMRW' # your Foursquare ID
CLIENT_SECRET = 'Z5RJ0TZLL00RWCH4TPYSFVCXFRO1VAXDMELH0MRAGPB4JUNI' # your Foursquare Secret
VERSION = '20210825' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PHSUFWQK30TO51RXWQO44GPOYGK1JEIVM1DKN1MRGNKOTMRW
CLIENT_SECRET:Z5RJ0TZLL00RWCH4TPYSFVCXFRO1VAXDMELH0MRAGPB4JUNI


In [39]:
df_toronto_borough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
3,M4E,East Toronto,The Beaches,43.67709,-79.29547
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306


#### 3.5 - Let's get all the venues by borough and neighborhood

In [40]:
def getNearbyVenues(boroughs, neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for borough, neighborhood, lat, lng in zip(boroughs, neighborhoods, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
borough_neighborhood_venues = getNearbyVenues(boroughs=df_toronto_borough['Borough'],
                                             neighborhoods=df_toronto_borough['Neighborhood'],
                                             latitudes=df_toronto_borough['Latitude'],
                                             longitudes=df_toronto_borough['Longitude'])

In [42]:
borough_neighborhood_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
2,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Berkeley Church,43.655123,-79.365873,Event Space


#### 3.6 - Checking the size of the dataframe

In [43]:
borough_neighborhood_venues.shape

(1749, 8)

Let's check how many venues were returned for each borough and neighborhood

In [44]:
borough_neighborhood_venues.groupby(['Borough', 'Neighborhood']).count()

Neighborhood Latitude  \
Borough                Neighborhood                                                                
Central Toronto        Davisville                                                             27   
                       Davisville North                                                        9   
                       Forest Hill North & West                                                2   
                       Lawrence Park                                                           2   
                       Moore Park, Summerhill East                                             2   
                       North Toronto West                                                      3   
                       Roselawn                                                                1   
                       Summerhill West, Rathnelly, South Hill, Forest ...                      6   
                       The Annex, North Midtown, Yorkville                                    23   
Downtown Toronto       Berczy Park                                                            63   
                       CN Tower, King and Spadina, Railway Lands, Harb...                     80   
                       Central Bay Street                                                     76   
                       Christie                                                               10   
                       Church and Wellesley                                                   85   
                       Commerce Court, Victoria Hotel                                        100   
                       First Canadian Place, Underground city                                100   
                       Garden District, Ryerson                                              100   
                       Harbourfront East, Union Station, Toronto Islands                      60   
                       Kensington Market, Chinatown, Grange Park                              59   
                       Regent Park, Harbourfront                                              22   
                       Richmond, Adelaide, King                                              100   
                       Rosedale                                                                4   
                       St. James Town                                                         87   
                       St. James Town, Cabbagetown                                            47   
                       Toronto Dominion Centre, Design Exchange                              100   
                       University of Toronto, Harbord                                         52   
Downtown Toronto Stn A Enclave of M5E                                                        100   
East Toronto           India Bazaar, The Beaches West                                         20   
                       Studio District                                                        50   
                       The Beaches                                                             4   
                       The Danforth West, Riverdale                                            7   
East Toronto Business  Enclave of M4L                                                        100   
East York/East Toronto The Danforth  East                                                      2   
West Toronto           Brockton, Parkdale Village, Exhibition Place                           85   
                       Dufferin, Dovercourt Village                                           20   
                       High Park, The Junction South                                           1   
                       Little Portugal, Trinity                                               45   
                       Parkdale, Roncesvalles                                                 56   
                       Runnymede, Swansea                                  

Let's find out how many unique categories can be curated from all the returned venues

In [45]:
print(f'There are {borough_neighborhood_venues["Venue Category"].nunique()} uniques categories.')

There are 225 uniques categories.


#### 3.7 - Analyze each Neighborhood and Venue

In [46]:
# one hot encoding
toronto_onehot = pd.get_dummies(borough_neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = borough_neighborhood_venues['Borough'] 
toronto_onehot['Neighborhoods'] = borough_neighborhood_venues['Neighborhood'] 
toronto_onehot['Venue'] = borough_neighborhood_venues['Venue'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Neighborhoods,Venue,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Downtown Toronto,"Regent Park, Harbourfront",Tandem Coffee,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,"Regent Park, Harbourfront",Roselle Desserts,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,"Regent Park, Harbourfront",Figs Breakfast & Lunch,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Downtown Toronto,"Regent Park, Harbourfront",The Yoga Lounge,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,Downtown Toronto,"Regent Park, Harbourfront",Berkeley Church,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
toronto_onehot.shape

(1749, 228)

Next, let's group rows by borough and neighborhood and by taking the mean of the frequency of occurrence of each category

In [48]:
toronto_grouped = toronto_onehot.groupby(['Borough', 'Neighborhoods']).mean().reset_index()
toronto_grouped

,Borough,Neighborhoods,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Central Toronto,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.037037,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
1,Central Toronto,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
2,Central Toronto,Forest Hill North & West,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
3,Central Toronto,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
4,Central Toronto,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
5,Central Toronto,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
6,Central Toronto,Roselawn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
7,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
8,Central Toronto,"The Annex, North Midtown, Yorkville",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
9,Downtown Toronto,Berczy Park,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.015873,0.000000,...,0.000000,0.00,0.000000,0.015873,0.000000,0.000000,0.000000,0.00,0.000000,0.015873


Let's confirm the new size

In [49]:
toronto_grouped.shape

(39, 227)

Let's print each neighborhood along with the top 5 most common venues

In [51]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0        Dessert Shop  0.11
1         Pizza Place  0.07
2  Italian Restaurant  0.07
3         Coffee Shop  0.07
4      Sandwich Place  0.07


----Davisville North----
                  venue  freq
0            Playground  0.11
1        Breakfast Spot  0.11
2  Gym / Fitness Center  0.11
3                   Gym  0.11
4   Japanese Restaurant  0.11


----Forest Hill North & West----
                        venue  freq
0           Accessories Store   0.5
1                        Park   0.5
2                   Nightclub   0.0
3          Miscellaneous Shop   0.0
4  Modern European Restaurant   0.0


----Lawrence Park----
                     venue  freq
0                 Bus Line   0.5
1              Swim School   0.5
2        Accessories Store   0.0
3  New American Restaurant   0.0
4       Miscellaneous Shop   0.0


----Moore Park, Summerhill East----
                        venue  freq
0                         Gym   1.0
1           Accessories

Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough', 'Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Davisville,Dessert Shop,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Pizza Place,Fast Food Restaurant,Farmers Market,Seafood Restaurant,Restaurant
1,Central Toronto,Davisville North,Japanese Restaurant,Gym / Fitness Center,Gym,Playground,Hotel,Park,Breakfast Spot,Department Store,Food & Drink Shop,Yoga Studio
2,Central Toronto,Forest Hill North & West,Accessories Store,Park,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
3,Central Toronto,Lawrence Park,Bus Line,Swim School,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
4,Central Toronto,"Moore Park, Summerhill East",Gym,Yoga Studio,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [54]:
neighborhoods_venues_sorted.shape

(39, 12)

#### 3.8 Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [55]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(['Borough', 'Neighborhoods'], 1)

# run k-means clustering
kmeans = KMeans(init='k-means++', n_clusters=kclusters, random_state=1, max_iter=1000).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 3, 4, 0, 3, 2, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [56]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

toronto_merged = df_toronto_borough.copy()

# merge toronto_grouped with df_toronto_borough to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(['Borough'], axis=1).set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Coffee Shop,Breakfast Spot,Bakery,Gym / Fitness Center,Distribution Center,Pub,Restaurant,Electronics Store,Event Space,Spa,1
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Hotel,Bubble Tea Shop,Diner,Italian Restaurant,1
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,Coffee Shop,Restaurant,Italian Restaurant,Cosmetics Shop,Café,Clothing Store,Cocktail Bar,Gym,Gastropub,Hotel,1
3,M4E,East Toronto,The Beaches,43.67709,-79.29547,Health Food Store,Neighborhood,Trail,Pub,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,1
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306,Coffee Shop,Seafood Restaurant,Bakery,Cheese Shop,Farmers Market,Pharmacy,Restaurant,Cocktail Bar,Beer Bar,Molecular Gastronomy Restaurant,1


Finally, let's visualize the resulting clusters

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### 3.9 - Examining the Clusters

Cluster - 1

In [59]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
29,Central Toronto,Gym,Yoga Studio,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,0


Cluster - 2

In [60]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Downtown Toronto,Coffee Shop,Breakfast Spot,Bakery,Gym / Fitness Center,Distribution Center,Pub,Restaurant,Electronics Store,Event Space,Spa,1
1,Downtown Toronto,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Hotel,Bubble Tea Shop,Diner,Italian Restaurant,1
2,Downtown Toronto,Coffee Shop,Restaurant,Italian Restaurant,Cosmetics Shop,Café,Clothing Store,Cocktail Bar,Gym,Gastropub,Hotel,1
3,East Toronto,Health Food Store,Neighborhood,Trail,Pub,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,1
4,Downtown Toronto,Coffee Shop,Seafood Restaurant,Bakery,Cheese Shop,Farmers Market,Pharmacy,Restaurant,Cocktail Bar,Beer Bar,Molecular Gastronomy Restaurant,1
5,Downtown Toronto,Coffee Shop,Clothing Store,Sushi Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Sandwich Place,Cosmetics Shop,Hotel,Bookstore,Bank,1
6,Downtown Toronto,Café,Grocery Store,Playground,Coffee Shop,Italian Restaurant,Baby Store,Candy Store,Yoga Studio,Ethiopian Restaurant,Electronics Store,1
7,Downtown Toronto,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gym,Asian Restaurant,Salad Place,Vegetarian / Vegan Restaurant,Steakhouse,1
8,West Toronto,Grocery Store,Park,Pharmacy,Bakery,Furniture / Home Store,Middle Eastern Restaurant,Pet Store,Gym / Fitness Center,Pizza Place,Post Office,1
10,Downtown Toronto,Coffee Shop,Hotel,Plaza,Japanese Restaurant,Deli / Bodega,Supermarket,Park,Boat or Ferry,Café,Restaurant,1


Cluster - 3

In [61]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
19,Central Toronto,Home Service,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,2


Cluster - 4

In [62]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
9,East York/East Toronto,Convenience Store,Park,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,3
21,Central Toronto,Accessories Store,Park,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,3
22,West Toronto,Park,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,3
23,Central Toronto,Gym Pool,Playground,Park,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,3
33,Downtown Toronto,Park,Playground,Bike Trail,Yoga Studio,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,3


Cluster - 5

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
18,Central Toronto,Bus Line,Swim School,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,4
